In [1]:
# Import the data
import pandas as pd
import numpy as np
import pydotplus
from IPython.display import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, accuracy_score, confusion_matrix, roc_auc_score
from sklearn.datasets import make_blobs
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.preprocessing import StandardScaler



In [2]:
population_df = pd.read_csv("population_by_year_2014_2024.csv")
meat_df = pd.read_csv("Consumption of meat per capita.csv")

In [5]:
population_df = population_df.drop(columns= ['2022', '2023', '2024'])
population_df.head()

,Country,2014,2015,2016,2017,2018,2019,2020,2021
0,Afghanistan,56805036,58070279,59363704,60685938,62037623,63419415,64831984,66276015
1,Albania,127571863,129443136,131341857,133268430,135223262,137206769,139219371,141261494
2,Algeria,147582820,151776865,156090098,160525905,165087770,169779276,174604106,179566049
3,Andorra,133274038,134398607,135532666,136676293,137829571,138992580,140165403,141348122
4,Angola,35838921,36323701,36815040,37313024,37817745,38329292,38847760,39373240


In [10]:
meat_df = meat_df[meat_df['Year'] >= 2014]
meat_df.head()

,Entity,Year,Poultry,Beef,Sheep and goat,Pork,Other meats,Fish and seafood
53,Afghanistan,2014,2.11,4.989999,5.059999,0.03,0.43,0.19
54,Afghanistan,2015,1.96,3.810000,4.770000,0.02,0.36,0.21
55,Afghanistan,2016,2.00,3.280000,4.800000,0.01,0.36,0.23
56,Afghanistan,2017,1.79,2.920000,4.450000,0.02,0.33,0.43
57,Afghanistan,2018,1.46,2.670000,4.319999,0.03,0.32,0.42


In [9]:
population_long = pd.melt(population_df, id_vars=["Country"],
                          var_name="Year", value_name="Population")
population_long.rename(columns={"Country": "Entity"}, inplace=True)
population_long["Year"] = pd.to_numeric(population_long["Year"])
population_long.head(500)

,Entity,Year,Population
0,Afghanistan,2014,56805036
1,Albania,2014,127571863
2,Algeria,2014,147582820
3,Andorra,2014,133274038
4,Angola,2014,35838921
...,...,...,...
495,Micronesia,2016,61844204
496,Moldova,2016,139568745
497,Monaco,2016,92573003
498,Mongolia,2016,50258012


In [13]:
population_long["Year"] = pd.to_numeric(population_long["Year"], errors='coerce')
meat_df["Year"] = pd.to_numeric(meat_df["Year"], errors='coerce')
combined_df = pd.merge(population_long, meat_df, on=["Entity", "Year"], how="inner")
combined_df.head()

,Entity,Year,Population,Poultry,Beef,Sheep and goat,Pork,Other meats,Fish and seafood
843,Barbados,2019,88889298,50.117672,14.429331,3.639831,9.749547,0.059997,42.618020
844,Belarus,2019,5233698,27.099997,19.239998,0.110000,37.369995,0.240000,11.739998
845,Belgium,2019,122244874,12.750002,14.500002,1.160000,31.690004,1.950000,24.350002
846,Belize,2019,3763328,27.359790,4.759963,0.079999,25.269806,0.279998,17.899862
847,Benin,2019,113741503,9.809999,3.509999,0.770000,0.700000,0.670000,16.589996
...,...,...,...,...,...,...,...,...,...
1338,Venezuela,2021,123746948,16.280005,13.020003,0.350000,5.940002,0.010000,10.290003
1339,Vietnam,2021,57561909,19.620003,5.180000,0.230000,27.300001,0.260000,39.500004
1340,Yemen,2021,105918565,10.739999,2.950000,3.940000,0.010000,0.150000,2.550000
1341,Zambia,2021,175108059,3.240000,10.299999,0.810000,1.870000,2.100000,12.329999
